In [1]:
!pip install -q faiss-cpu langchain langchain-community langchain-experimental langchain-cohere

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 60.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 89.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.2/209.2 kB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 291.9/291.9 kB 24.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 91.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 5.0 MB/s eta 0:00:00


In [2]:
# @title Imports

import os
import re
import requests
import numpy as np
import pandas as pd
from google.colab import userdata
from IPython.display import display

# LangChain & LangChain Community
import faiss
from langchain.prompts import PromptTemplate
from langchain.docstore.document import Document
from langchain.vectorstores.faiss import FAISS
from langchain.chains import LLMChain, RetrievalQA, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_experimental.agents import create_pandas_dataframe_agent

In [3]:
# @title Download excel files

headers = {
    'User-Agent': 'microsoft bill.gates@microsoft.com',
    'Accept': 'application/vnd.openxmlformats-officedocument.spreadsheetml.sheet'
}

response = requests.get("https://www.sec.gov/Archives/edgar/data/0000320193/000032019321000105/Financial_Report.xlsx", headers=headers)
with open("/content/Apple_Financial_2021.xlsx", 'wb') as f:
    f.write(response.content)

response = requests.get("https://www.sec.gov/Archives/edgar/data/0001682852/000168285225000022/Financial_Report.xlsx", headers=headers)
with open("/content/Moderna_Financial_2025.xlsx", 'wb') as f:
    f.write(response.content)

response = requests.get("https://www.sec.gov/Archives/edgar/data/0000064803/000006480324000007/Financial_Report.xlsx", headers=headers)
with open("/content/CVS_Health_Financial_2024.xlsx", 'wb') as f:
    f.write(response.content)

response = requests.get("https://www.sec.gov/Archives/edgar/data/0001318605/000095017023001409/Financial_Report.xlsx", headers=headers)
with open("/content/Tesla_Financial_2023.xlsx", 'wb') as f:
    f.write(response.content)

In [4]:
# @title excel_to_df(excel_file_path: str) - Convert excel data to dataframe
def excel_to_df(excel_file_path: str):
    sheet_name_desc1 = 'consolidated statements of oper'
    sheet_name_desc2 = 'consolidated statements of incom'
    global files_parsed, files_not_parsed
    try:
        matched_sheet = ''
        excel_content = pd.ExcelFile(excel_file_path)
        for sheet_name in excel_content.sheet_names:
            if sheet_name_desc1 in sheet_name.lower() or sheet_name_desc2 in sheet_name.lower():
              matched_sheet = sheet_name
              break
        df = pd.read_excel(excel_file_path, sheet_name = matched_sheet, engine = 'openpyxl')
        files_parsed.append(excel_file_path)
    except Exception as e:
        # If df is not found then return 'Required sheet not found!'
        df = f'Could not read the excel file {excel_file_path}!'
        files_not_parsed.append(excel_file_path)

    return df

In [5]:
# @title def process_df_values(df: pd.dataframe, share_multiplier: int, usd_multiplier: int) -> Correct the values in dataframe according to share and usd multipliers

def process_df_values(df: pd.DataFrame, share_multiplier: int, usd_multiplier: int) -> pd.DataFrame:
    pre_string = ''
    nan_index, text_index = [], []

    common_averages = ["in share", "average share", "average common share"]
    usd_terms = ["in usd", "in dollars", "income per share"]

    for index, row in df.iterrows():
        try:
            row_desc = row.loc['Description'].lower() # Get the row description
            if row_desc[-1] == ":":
                row_desc = row_desc[:-1]
        except:
            continue

        if row.isnull().all():
          pre_string = ''
          nan_index.append(index)

        elif row.iloc[2:].isnull().all():
          if index - 1 in nan_index:
            pre_string = pre_string + " | " + row_desc #re.sub(r'[^a-zA-Z]+$[]', '', row_desc)
          else:
            pre_string = row_desc #re.sub(r'[^a-zA-Z]+$[]', '', row_desc)
          nan_index.append(index)

        else:
            # If value can't be converted to float, then skip that row
            try:
                float(row.iloc[2])

                if pre_string:
                    df.iloc[index, 0] = pre_string

                for col in df.columns[2:]:
                    val = row[col]
                    if any(common_average in row_desc for common_average in common_averages) or any(common_average in pre_string for common_average in common_averages):
                        val = float(val) * share_multiplier
                        # val = str(val) + " shares"
                    else:
                        if all(usd_term not in row_desc and usd_term not in pre_string for usd_term in usd_terms):
                            val = float(val) * usd_multiplier
                            # if val >= 1000000000:
                            #     val = str(val/1000000000) + " billion"
                            # elif val >= 1000000:
                            #     val = str(val/1000000) + " million"

                        # val = "USD " + str(val)

                    df.at[index, col] = val
            except:
                text_index.append(index)
                continue

    if text_index:
      df = df.drop(text_index)

    df = df.dropna(subset=df.columns[1:])

    return df.reset_index(drop = True)

In [6]:
# @title def process_dataframe(df: pd.dataframe) -> Set column headings and find share and usd multipliers

def process_dataframe(df: pd.DataFrame) -> pd.DataFrame:
    multiplier_identifier = df.columns[0].lower()

    share_multiplier = 1
    if 'shares in billion' in multiplier_identifier:
        share_multiplier = 1000000000
    elif 'shares in million' in multiplier_identifier:
        share_multiplier = 1000000
    elif 'shares in thousand' in multiplier_identifier:
        share_multiplier = 1000

    usd_multiplier = 1
    if '$ in billion' in multiplier_identifier:
        usd_multiplier = 1000000000
    elif '$ in million' in multiplier_identifier:
        usd_multiplier = 1000000
    elif '$ in thousand' in multiplier_identifier:
        usd_multiplier = 1000

    df.columns = [f'new_column_{i}' if col is None else col for i, col in enumerate(df.columns)]

    df = df.map(lambda x: np.nan if (x == '' or x is None or (isinstance(x, str) and x.strip() == "")) else x)

    for column in df.columns:
        if df[column].nunique() < 5:
            df = df.drop(columns=[column])

    df = df.rename(columns = {df.columns[0]: "Description",
                              df.columns[1]: int(df.iloc[0,1][-4:]),
                              df.columns[2]: int(df.iloc[0,2][-4:]),
                              df.columns[3]: int(df.iloc[0,3][-4:])})

    df = df[1:]
    df.insert(0, 'Category', '')
    df = df.reset_index(drop = True)      # To make sure indices are in sync

    return process_df_values(df = df, share_multiplier = share_multiplier, usd_multiplier = usd_multiplier)


In [7]:
# @title def df_to_document(df:  pd.DataFrame, company_name: str) -> Convert dataframe to langchain Documents
def df_to_document(df:  pd.DataFrame, company_name: str):
    documents = []

    for index, row in df.iterrows():
      if row.iloc[0]:
        # row_desc = f"{row.iloc[1]} in {row.iloc[0]} category"
        row_desc = f"For {company_name} {row.iloc[1]} in {row.iloc[0]} category for year"
      else:
        # row_desc = f"{row.iloc[1]}"
        row_desc = f"For {company_name} {row.iloc[1]} for year"

      for column in df.columns[2:]:
        metadata = {"company": company_name, "year": column}
        documents.append(Document(page_content = f"{row_desc} {column} is {row[column]}.", metadata = metadata))

    return documents

In [8]:
# @title MAIN

file_list = ['/content/Apple_Financial_2021.xlsx', '/content/Moderna_Financial_2025.xlsx', '/content/CVS_Health_Financial_2024.xlsx', '/content/Tesla_Financial_2023.xlsx']
files_parsed, files_not_parsed = [], []

docs = []

# Convert excel to dataframe
for excel_file_path in file_list:

    company_name = (excel_file_path.split('/')[-1]).split('_')[0]
    df = excel_to_df(excel_file_path = excel_file_path )
    if not isinstance(df, pd.DataFrame):
        files_not_parsed.append(excel_file_path)
        continue

    files_parsed.append(excel_file_path)
    df = process_dataframe(df = df)
    display(df)
    print("\n\n")
    docs.extend(df_to_document(df = df, company_name = company_name))


,Category,Description,2021,2020,2019
0,,Net sales,365817000000.0,274515000000.0,260174000000.0
1,,Cost of sales,212981000000.0,169559000000.0,161782000000.0
2,,Gross margin,152836000000.0,104956000000.0,98392000000.0
3,operating expenses,Research and development,21914000000.0,18752000000.0,16217000000.0
4,operating expenses,"Selling, general and administrative",21973000000.0,19916000000.0,18245000000.0
5,operating expenses,Total operating expenses,43887000000.0,38668000000.0,34462000000.0
6,operating expenses,Operating income,108949000000.0,66288000000.0,63930000000.0
7,operating expenses,"Other income/(expense), net",258000000.0,803000000.0,1807000000.0
8,operating expenses,Income before provision for income taxes,109207000000.0,67091000000.0,65737000000.0
9,operating expenses,Provision for income taxes,14527000000.0,9680000000.0,10481000000.0


,Category,Description,2024,2023,2022
0,revenue,Total revenue,3236000000.0,6848000000.0,19263000000.0
1,operating expenses,Cost of sales,1464000000.0,4693000000.0,5416000000.0
2,operating expenses,Research and development,4543000000.0,4845000000.0,3295000000.0
3,operating expenses,"Selling, general and administrative",1174000000.0,1549000000.0,1132000000.0
4,operating expenses,Total operating expenses,7181000000.0,11087000000.0,9843000000.0
5,operating expenses,(Loss) income from operations,-3945000000.0,-4239000000.0,9420000000.0
6,operating expenses,Interest income,425000000.0,421000000.0,200000000.0
7,operating expenses,"Other expense, net",-87000000.0,-124000000.0,-45000000.0
8,operating expenses,(Loss) income before income taxes,-3607000000.0,-3942000000.0,9575000000.0
9,operating expenses,(Benefit from) provision for income taxes,-46000000.0,772000000.0,1213000000.0


,Category,Description,2023,2022,2021
0,revenues,Premiums,99192000000.0,85330000000.0,76132000000.0
1,revenues,Net investment income,1153000000.0,838000000.0,1199000000.0
2,revenues,Total revenues,357776000000.0,322467000000.0,292111000000.0
3,operating costs,Cost of products sold,217098000000.0,196892000000.0,175803000000.0
4,operating costs,Health care costs,86247000000.0,71073000000.0,64188000000.0
5,operating costs,Restructuring charges,507000000.0,0.0,0.0
6,operating costs,Opioid litigation charges,0.0,5803000000.0,0.0
7,operating costs,Loss on assets held for sale,349000000.0,2533000000.0,0.0
8,operating costs,Store impairments,0.0,0.0,1358000000.0
9,operating costs,Goodwill impairment,0.0,0.0,431000000.0


,Category,Description,2022,2021,2020
0,revenues,Total revenues,81462000000.0,53823000000.0,31536000000.0
1,cost of revenues,Total cost of revenues,60609000000.0,40217000000.0,24906000000.0
2,cost of revenues,Gross profit,20853000000.0,13606000000.0,6630000000.0
3,operating expenses,Research and development,3075000000.0,2593000000.0,1491000000.0
4,operating expenses,"Selling, general and administrative",3946000000.0,4517000000.0,3145000000.0
5,operating expenses,Restructuring and other,176000000.0,-27000000.0,0.0
6,operating expenses,Total operating expenses,7197000000.0,7083000000.0,4636000000.0
7,operating expenses,Income from operations,13656000000.0,6523000000.0,1994000000.0
8,operating expenses,Interest income,297000000.0,56000000.0,30000000.0
9,operating expenses,Interest expense,-191000000.0,-371000000.0,-748000000.0


In [10]:
# @title This cell restarts this notebook. Re-run all cells after restart
!pip uninstall -y langchain-cohere
!pip install -q "langchain-cohere>=0.1.0,<0.2.0"

Found existing installation: langchain-cohere 0.4.4
Uninstalling langchain-cohere-0.4.4:
  Successfully uninstalled langchain-cohere-0.4.4
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 397.1/397.1 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.2/207.2 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 41.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.8/311.8 kB 24.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 64.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 51.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
opencv-contrib-python 4.12.0.88 requires numpy<2.3.0,

In [11]:
# @title Embedding and Cohere Configuration

# Cohere integrations
from langchain_cohere import ChatCohere
from langchain_cohere import CohereEmbeddings
from langchain_community.llms import Cohere as LangChainCohere

cohere_api_key = userdata.get('COHERE_API_KEY')
embeddings = CohereEmbeddings(cohere_api_key=cohere_api_key, user_agent="langchain")

vectorstore = FAISS.from_documents(docs, embeddings)
retriever = vectorstore.as_retriever(search_kwargs={"k": 15})

llm = ChatCohere(cohere_api_key=cohere_api_key)

qa_chain = RetrievalQA.from_chain_type(llm=llm, retriever=retriever, return_source_documents=True)

PydanticUserError: The `__modify_schema__` method is not supported in Pydantic v2. Use `__get_pydantic_json_schema__` instead in class `SecretStr`.

For further information visit https://errors.pydantic.dev/2.11/u/custom-json-schema

In [ ]:
# @title Query 1 to Cohere

# This query works good
query = "Compare Moderna and Tesla's net income"
response = qa_chain({"query": query})

print("\nAnswer:")
print(response['result'])

In [ ]:
# @title Query 2 to Cohere

# This query does not work good
query = "What was CVS's net income in 2022?"
response = qa_chain({"query": query})

print("\nAnswer:")
print(response['result'])